In [107]:
import torch
from safetensors.torch import load_file, save_file, _remove_duplicate_names
from pprint import pprint
import pickle
import numpy as np

In [122]:
original_ckpt_path = "/home/thomwolf/Documents/Github/ACT/checkpoints/blue_red_sort_raw/policy_initial_state.ckpt"
stats_path = original_ckpt_path.replace('policy_initial_state.ckpt', f'dataset_stats.pkl')
converted_ckpt_path = "/home/thomwolf/Documents/Github/ACT/checkpoints/blue_red_sort_raw/initial_state/model.safetensors"

comparison_main_path = "/home/thomwolf/Documents/Github/lerobot/examples/real_robot_example/outputs/train/blue_red_debug_no_masking/checkpoints/last/pretrained_model/"
comparison_safetensor_path = comparison_main_path + "model.safetensors"
comparison_config_json_path = comparison_main_path + "config.json"
comparison_config_yaml_path = comparison_main_path + "config.yaml"

In [123]:

with open(stats_path, 'rb') as f:
    stats = pickle.load(f)
image_stats_mean = torch.tensor([0.485, 0.456, 0.406])
image_stats_std = torch.tensor([0.229, 0.224, 0.225])

In [124]:
stats

{'action_mean': array([ 0.10564873, -0.06760169, -0.21794759,  1.4932278 ,  0.01712005,
         0.65135974], dtype=float32),
 'action_std': array([0.324479  , 0.48200357, 0.6450592 , 0.21122734, 0.16231349,
        0.46708518], dtype=float32),
 'qpos_mean': array([ 0.10348293, -0.01160635, -0.06582164,  1.490679  ,  0.01592216,
         0.73001933], dtype=float32),
 'qpos_std': array([0.32309636, 0.4455181 , 0.59022886, 0.17841513, 0.16096668,
        0.32320786], dtype=float32),
 'example_qpos': array([[ 0.05212891, -0.58875   ,  0.76200193,  1.4289453 , -0.0015332 ,
          0.9827832 ],
        [ 0.05212891, -0.58875   ,  0.76200193,  1.4304786 , -0.0015332 ,
          0.9827832 ],
        [ 0.05212891, -0.58875   ,  0.76200193,  1.4304786 , -0.0015332 ,
          0.9827832 ],
        ...,
        [-0.03066406, -0.36030275,  0.7190723 ,  1.1897656 , -0.18398437,
          0.9751172 ],
        [-0.03066406, -0.35876954,  0.72367185,  1.1897656 , -0.18398437,
          0.9751172 ],


In [125]:
a = torch.load(original_ckpt_path)

In [126]:
b = load_file(comparison_safetensor_path)

In [127]:
dest = list(b.keys())
pprint(dest)

['model.action_head.bias',
 'model.action_head.weight',
 'model.backbone.bn1.bias',
 'model.backbone.bn1.running_mean',
 'model.backbone.bn1.running_var',
 'model.backbone.bn1.weight',
 'model.backbone.conv1.weight',
 'model.backbone.layer1.0.bn1.bias',
 'model.backbone.layer1.0.bn1.running_mean',
 'model.backbone.layer1.0.bn1.running_var',
 'model.backbone.layer1.0.bn1.weight',
 'model.backbone.layer1.0.bn2.bias',
 'model.backbone.layer1.0.bn2.running_mean',
 'model.backbone.layer1.0.bn2.running_var',
 'model.backbone.layer1.0.bn2.weight',
 'model.backbone.layer1.0.conv1.weight',
 'model.backbone.layer1.0.conv2.weight',
 'model.backbone.layer1.1.bn1.bias',
 'model.backbone.layer1.1.bn1.running_mean',
 'model.backbone.layer1.1.bn1.running_var',
 'model.backbone.layer1.1.bn1.weight',
 'model.backbone.layer1.1.bn2.bias',
 'model.backbone.layer1.1.bn2.running_mean',
 'model.backbone.layer1.1.bn2.running_var',
 'model.backbone.layer1.1.bn2.weight',
 'model.backbone.layer1.1.conv1.weight',


In [128]:
orig = list(a.keys())
pprint(orig)

['model.pos_table',
 'model.transformer.encoder.layers.0.self_attn.in_proj_weight',
 'model.transformer.encoder.layers.0.self_attn.in_proj_bias',
 'model.transformer.encoder.layers.0.self_attn.out_proj.weight',
 'model.transformer.encoder.layers.0.self_attn.out_proj.bias',
 'model.transformer.encoder.layers.0.linear1.weight',
 'model.transformer.encoder.layers.0.linear1.bias',
 'model.transformer.encoder.layers.0.linear2.weight',
 'model.transformer.encoder.layers.0.linear2.bias',
 'model.transformer.encoder.layers.0.norm1.weight',
 'model.transformer.encoder.layers.0.norm1.bias',
 'model.transformer.encoder.layers.0.norm2.weight',
 'model.transformer.encoder.layers.0.norm2.bias',
 'model.transformer.encoder.layers.1.self_attn.in_proj_weight',
 'model.transformer.encoder.layers.1.self_attn.in_proj_bias',
 'model.transformer.encoder.layers.1.self_attn.out_proj.weight',
 'model.transformer.encoder.layers.1.self_attn.out_proj.bias',
 'model.transformer.encoder.layers.1.linear1.weight',
 '

In [129]:
a = torch.load(original_ckpt_path)

to_remove_startswith = ['model.transformer.decoder.layers.1.',
             'model.transformer.decoder.layers.2.',
             'model.transformer.decoder.layers.3.',
             'model.transformer.decoder.layers.4.',
             'model.transformer.decoder.layers.5.',
             'model.transformer.decoder.layers.6.',
             'model.transformer.decoder.norm.',
             'model.is_pad_head']

to_remove_in = ['num_batches_tracked',]

conv = {}

keys = list(a.keys())
for k in keys:
    if any(k.startswith(tr) for tr in to_remove_startswith):
        a.pop(k)
        continue
    if any(tr in k for tr in to_remove_in):
        a.pop(k)
        continue
    if k.startswith('model.transformer.encoder.layers.'):
        conv[k.replace('transformer.', '')] = a.pop(k)
    if k.startswith('model.transformer.decoder.layers.0.'):
        conv[k.replace('transformer.', '')] = a.pop(k)
    if k.startswith('model.encoder.layers.'):
        conv[k.replace('encoder.', 'vae_encoder.')] = a.pop(k)
    if k.startswith('model.action_head.'):
        conv[k] = a.pop(k)
    if k.startswith('model.pos_table'):
        conv[k.replace('pos_table', 'vae_encoder_pos_enc')] = a.pop(k)
    if k.startswith('model.query_embed.'):
        conv[k.replace('query_embed', 'decoder_pos_embed')] = a.pop(k)
    if k.startswith('model.input_proj.'):
        conv[k.replace('input_proj.', 'encoder_img_feat_input_proj.')] = a.pop(k)
    if k.startswith('model.input_proj_robot_state.'):
        conv[k.replace('input_proj_robot_state.', 'encoder_robot_state_input_proj.')] = a.pop(k)
    if k.startswith('model.backbones.0.0.body.'):
        conv[k.replace('backbones.0.0.body', 'backbone')] = a.pop(k)
    if k.startswith('model.cls_embed.'):
        conv[k.replace('cls_embed', 'vae_encoder_cls_embed')] = a.pop(k)
    if k.startswith('model.encoder_action_proj.'):
        conv[k.replace('encoder_action_proj', 'vae_encoder_action_input_proj')] = a.pop(k)
    if k.startswith('model.encoder_joint_proj.'):
        conv[k.replace('encoder_joint_proj', 'vae_encoder_robot_state_input_proj')] = a.pop(k)
    if k.startswith('model.latent_proj.'):
        conv[k.replace('latent_proj', 'vae_encoder_latent_output_proj')] = a.pop(k)
    if k.startswith('model.latent_out_proj.'):
        conv[k.replace('latent_out_proj', 'encoder_latent_input_proj')] = a.pop(k)
    if k.startswith('model.additional_pos_embed.'):
        conv[k.replace('additional_pos_embed', 'encoder_robot_and_latent_pos_embed')] = a.pop(k)

In [130]:
conv['normalize_inputs.buffer_observation_images_front.mean'] = image_stats_mean[:, None, None]
conv['normalize_inputs.buffer_observation_images_front.std'] = image_stats_std[:, None, None]
conv['normalize_inputs.buffer_observation_images_top.mean'] = image_stats_mean[:, None, None].clone()
conv['normalize_inputs.buffer_observation_images_top.std'] = image_stats_std[:, None, None].clone()

conv['normalize_inputs.buffer_observation_state.mean'] = torch.tensor(stats['qpos_mean'])
conv['normalize_inputs.buffer_observation_state.std'] = torch.tensor(stats['qpos_std'])
conv['normalize_targets.buffer_action.mean'] = torch.tensor(stats['action_mean'])
conv['normalize_targets.buffer_action.std'] = torch.tensor(stats['action_std'])

conv['unnormalize_outputs.buffer_action.mean'] = torch.tensor(stats['action_mean'])
conv['unnormalize_outputs.buffer_action.std'] = torch.tensor(stats['action_std'])

In [131]:
a

OrderedDict()

In [132]:
not_converted = set(b.keys())
for k, v in conv.items():
    try:
        b[k].shape == v.squeeze().shape
    except Exception as e:
        print(k, v)
        print(b[k].shape)
        print(e)
    b[k] = v
    not_converted.remove(k)

In [133]:
not_converted

set()

In [134]:
metadata = None
to_removes = _remove_duplicate_names(b)
print(to_removes)

defaultdict(<class 'list'>, {})


In [135]:
for kept_name, to_remove_group in to_removes.items():
    for to_remove in to_remove_group:
        if metadata is None:
            metadata = {}

        if to_remove not in metadata:
            # Do not override user data
            metadata[to_remove] = kept_name
        del b[to_remove]
save_file(b, converted_ckpt_path)

In [136]:
# Now also copy the config files
import shutil
shutil.copy(comparison_config_json_path, converted_ckpt_path.replace('model.safetensors', 'config.json'))
shutil.copy(comparison_config_yaml_path, converted_ckpt_path.replace('model.safetensors', 'config.yaml'))

'/home/thomwolf/Documents/Github/ACT/checkpoints/blue_red_sort_raw/initial_state/config.yaml'

In [137]:
c = load_file(converted_ckpt_path)
c.keys()

dict_keys(['model.action_head.bias', 'model.action_head.weight', 'model.backbone.bn1.bias', 'model.backbone.bn1.running_mean', 'model.backbone.bn1.running_var', 'model.backbone.bn1.weight', 'model.backbone.conv1.weight', 'model.backbone.layer1.0.bn1.bias', 'model.backbone.layer1.0.bn1.running_mean', 'model.backbone.layer1.0.bn1.running_var', 'model.backbone.layer1.0.bn1.weight', 'model.backbone.layer1.0.bn2.bias', 'model.backbone.layer1.0.bn2.running_mean', 'model.backbone.layer1.0.bn2.running_var', 'model.backbone.layer1.0.bn2.weight', 'model.backbone.layer1.0.conv1.weight', 'model.backbone.layer1.0.conv2.weight', 'model.backbone.layer1.1.bn1.bias', 'model.backbone.layer1.1.bn1.running_mean', 'model.backbone.layer1.1.bn1.running_var', 'model.backbone.layer1.1.bn1.weight', 'model.backbone.layer1.1.bn2.bias', 'model.backbone.layer1.1.bn2.running_mean', 'model.backbone.layer1.1.bn2.running_var', 'model.backbone.layer1.1.bn2.weight', 'model.backbone.layer1.1.conv1.weight', 'model.backbone

In [105]:
c['normalize_inputs.buffer_observation_images_front.mean']

tensor([[[0.4850]],

        [[0.4560]],

        [[0.4060]]])